In [2]:
import pandas as pd
import numpy as np
import math
from operator import itemgetter
import csv

In [161]:
data=pd.read_csv(r'F:\work\Downloads\query-hive-532743.csv')
data

essay_id                                          embedding
0        290607  [-0.03454599415204088, -0.05767892252994755, -...
1        394127  [0.021726436906913695, -0.14967767599556367, 0...
2        504847  [-0.043751195395914474, -0.1801987305848531, 0...
3        535607  [-0.04687327534235502, -0.0788246007981876, 0....
4        558627  [-0.15998830875424164, -0.008907250447437556, ...
...         ...                                                ...
13928   3975615  [0.011311679953602665, -0.04959526588619834, 0...
13929   3975738  [0.20882117277040552, 0.03772823049046935, 0.0...
13930   3975659  [0.06788959363034361, -0.05716772578525943, -0...
13931   3976748  [-0.000640036019327188, 0.003873376695812714, ...
13932   3976835  [-0.13987608469465387, -0.16258167649428337, -...

[13933 rows x 2 columns]

In [3]:
effective_item_dict={}
for i in data.index:
    word=data['essay_id'][i]
    embedding=data['embedding'][i].replace('[','').replace(']','').split(', ')
    embedding=[float(x) for x in embedding]
    effective_item_dict[word]=embedding
print('有效文章向量存储成功')
effective_matrix=np.zeros((len(effective_item_dict.keys()),100))
effective_lst=list(effective_item_dict.keys())
for i in range(len(effective_lst)):
    effective_matrix[i]=np.array(effective_item_dict[effective_lst[i]])
print('有效文章向量矩阵存储成功')


有效文章向量存储成功
有效文章向量矩阵存储成功


In [4]:
data=pd.read_csv(r'F:\work\Downloads\query-hive-532782.csv')
data

essay_id                                          embedding
0        165987  [-0.05439096034671735, 0.0508400606827551, 0.0...
1        171487  [-0.047201549624971126, -0.10145030526854272, ...
2        176787  [-0.015890114888584925, 0.058667642409633666, ...
3        197427  [0.030642318547615323, 0.0020018767984995144, ...
4        197947  [0.13634151446800868, 0.030653411269028142, 0....
...         ...                                                ...
41018   3976435  [0.026192074422462217, -0.05536739620745684, 0...
41019   3976820  [-0.11907232120608446, -0.19200648090299086, -...
41020   3976748  [-0.000640036019327188, 0.003873376695812714, ...
41021   3976835  [-0.13987608469465387, -0.16258167649428337, -...
41022   3976717  [-0.0031942413866304243, 0.07136006422161735, ...

[41023 rows x 2 columns]

In [5]:
act_item_dict={}
for i in data.index:
    word=data['essay_id'][i]
    embedding=data['embedding'][i].replace('[','').replace(']','').split(', ')
    embedding=[float(x) for x in embedding]
    act_item_dict[word]=embedding
print('有行为的文章向量存储成功')
act_matrix=np.zeros((len(act_item_dict.keys()),100))
act_lst=list(act_item_dict.keys())
for i in range(len(act_lst)):
    act_matrix[i]=np.array(act_item_dict[act_lst[i]])
print('有行为的文章向量矩阵存储成功')

有行为的文章向量存储成功
有行为的文章向量矩阵存储成功


In [6]:
sim_matrix=np.dot(act_matrix,effective_matrix.T)
print('相似度矩阵计算成功')

相似度矩阵计算成功


In [7]:
sim_matrix

array([[0.28474357, 0.40672438, 0.41085044, ..., 0.29364577, 0.48139597,
        0.36697867],
       [0.3112797 , 0.44388688, 0.4604408 , ..., 0.57788926, 0.5480703 ,
        0.64997356],
       [0.33867384, 0.33343447, 0.40530145, ..., 0.59076332, 0.67884998,
        0.40865089],
       ...,
       [0.33047738, 0.35174861, 0.36804886, ..., 0.5395158 , 1.        ,
        0.37340092],
       [0.2447896 , 0.36504686, 0.38979845, ..., 0.35721614, 0.37340092,
        1.        ],
       [0.3884289 , 0.48645584, 0.49145097, ..., 0.4674928 , 0.72388667,
        0.49231056]])

In [8]:
output={}
for i in range(len(act_lst)):
    act_item=act_lst[i]
    output.setdefault(act_item,{})
    act_tensor=sim_matrix[i]
    sorted_list=sorted(enumerate(list(act_tensor)), key=itemgetter(1),reverse=True)
    for j,sim in sorted_list[1:51]:
        sim_item=effective_lst[j]
        output[act_item][sim_item]=sim
print('相似度获取成功!')

相似度获取成功!


In [9]:
import timeit
from sklearn.neighbors import KDTree,BallTree
kd_tree=KDTree(effective_matrix,metric='euclidean')
ball_tree=BallTree(effective_matrix,metric='euclidean')

In [10]:
from annoy import AnnoyIndex
d=100
ann_index=AnnoyIndex(d,'euclidean')
n=len(effective_matrix)
for i in range(n):
    ann_index.add_item(i,effective_matrix[i])
ann_index.build(100)

True

In [11]:
import hnswlib
num_elements=effective_matrix.shape[0]
dim=effective_matrix.shape[1]
p=hnswlib.Index(space='l2',dim=dim)
p.init_index(max_elements=num_elements,ef_construction=100,M=16)
p.set_ef(10)
p.set_num_threads(4)
p.add_items(effective_matrix)

In [18]:
def test_kdtree(x_emb,kd_tree):
    x_emb=x_emb.reshape(1,-1)
    kd_tree.query(x_emb,k=10,return_distance=False)
def test_balltree(x_emb,ball_tree):
    x_emb=x_emb.reshape(1,-1)
    ball_tree.query(x_emb,k=10,return_distance=False)
def test_annoy(x_emb,ann_index):
    ann_index.get_nns_by_vector(x_emb,10,include_distances=False)
def test_hnsw(x_emb,p):
    p.knn_query(x_emb,10)
def test_time(sql):
    m = timeit.repeat(sql, repeat=100, number=1, globals=globals())
    m = np.array(m) * 1000.0 #换算为毫秒
    print(np.min(m), np.mean(m), np.max(m), np.std(m))

In [19]:
test_time("test_kdtree(effective_matrix[0],kd_tree)")
test_time("test_balltree(effective_matrix[0],kd_tree)")
test_time("test_annoy(effective_matrix[0],ann_index)")
test_time("test_hnsw(effective_matrix[0],p)")

3.05160000016258 4.439134000015201 11.365499999783424 0.9848903477113952
2.9211999999461113 4.0201630000024124 7.298099999843544 0.6742909289946108
0.1752000000578846 0.23832499999571155 1.2807999999040476 0.11953822265101184
0.02599999993435631 0.03934499999104446 0.8433999998942454 0.08126162608334471


In [14]:
#ball_tree
x_emb=effective_matrix[0]
x_emb=x_emb.reshape(1,-1)
ball_tree.query(x_emb,k=10,return_distance=True)

(array([[0.        , 0.44473956, 0.44567583, 0.48275337, 0.48384814,
         0.5045995 , 0.51051476, 0.51127899, 0.51776073, 0.53016952]]),
 array([[    0, 13197,  6944,  6965,  4713,  4122,  8523,   699,  2312,
          2837]], dtype=int64))

In [46]:
#kd_tree
x_emb=effective_matrix[0]
x_emb=x_emb.reshape(1,-1)
kd_tree.query(x_emb,k=10,return_distance=True)

(array([[0.        , 0.44473956, 0.44567583, 0.48275337, 0.48384814,
         0.5045995 , 0.51051476, 0.51127899, 0.51776073, 0.53016952]]),
 array([[    0, 13197,  6944,  6965,  4713,  4122,  8523,   699,  2312,
          2837]], dtype=int64))

In [16]:
x_emb=effective_matrix[0]
ann_index.get_nns_by_vector(x_emb,10,include_distances=True)

([0, 13197, 6944, 6965, 4713, 4122, 8523, 699, 2312, 2837],
 [0.0,
  0.44473958015441895,
  0.445675790309906,
  0.48275336623191833,
  0.4838481545448303,
  0.5045995116233826,
  0.5105147361755371,
  0.5112789869308472,
  0.5177607536315918,
  0.5301695466041565])

In [17]:
x_emb=effective_matrix[0]
p.knn_query(x_emb,10)

(array([[    0, 13197,  6944,  6965,  4713,  4122,  8523,   699,  2312,
          2837]], dtype=uint64),
 array([[0.        , 0.19779329, 0.19862692, 0.23305082, 0.23410903,
         0.25462064, 0.2606253 , 0.2614062 , 0.26807618, 0.28107974]],
       dtype=float32))

In [139]:
user_data=pd.read_csv(r'F:\work\Downloads\query-hive-534786.csv')
user_data

deviceid  \
0     84960a1774a3460c93c5b010b4b97cef   
1     849628ecb8124c3fa0bc3252d7898f05   
2     8496d6c69fa8b53e780022e863c6b923   
3     84977bcb0b657af074298725e884c289   
4     8497be5892c6545afb11f16dcf76c832   
...                                ...   
9995  9a164075725141d48507a86a4a845e23   
9996  9a16a1cc4adc4c538652ff2db5a1a419   
9997  9a17976c56b8223b1a6f2107cc8bfc56   
9998  9a1828c931b44aadd4f415375918bed4   
9999  9a190b58345b0a51a9b815293cedd437   

                                      user_embedding_kw  
0     0.10915773219937779,0.23889515323733762,-0.090...  
1     0.19140144773443826,0.03375510105250599,-0.028...  
2     0.05954629433182673,0.012522418547905638,0.115...  
3     0.05924593841824724,0.09227478363833543,-0.005...  
4     0.10131438849453007,0.12851033599115833,0.0211...  
...                                                 ...  
9995  0.17791186277926466,0.08001076235345247,-0.045...  
9996  0.09748168013091646,0.11708576301788566,-0.032...  
9997  0.11760675324377234,0.08334436103322061,-0.005...  
9998  0.08200259022899334,0.14867224005763136,0.0273...  
9999  0.07268666901445894,-0.017808301998724005,-0.0...  

[10000 rows x 2 columns]

In [140]:
user_dict={}
for i in user_data.index:
    deviceid=user_data['deviceid'][i]
    embedding=user_data['user_embedding_kw'][i].split(',')
    embedding=[float(x) for x in embedding]
    user_dict[deviceid]=embedding

In [163]:
titles=pd.read_csv(r'F:\work\Downloads\query-hive-534377.csv')
titles

essay_id                      e.c_title
0         98511                2018国庆节摩旅太行山岳家寨
1        140622                   摩托骑行泰安大汶口明石桥
2        149412    海南有多美！临高县一日游！！！（喜欢的朋友点个赞！😄）
3        185876                            助推器
4        186680  你扔掉的摩托车轮胎， 在非洲肯尼亚居然做成鞋子了，愣住了！
...         ...                            ...
14339   3987085             逆天颜值 胡思瓦娜诺顿901即将量产
14340   3987099       众人皆草木,唯你是青山 ，机车女神（无水印高清）
14341   3987119                 贝纳利 唯一款六缸车长什么样
14342   3987138        所爱隔山海，山海不可平，机车女神（无水印高清）
14343   3987144                          无聊溜一溜

[14344 rows x 2 columns]

In [72]:
title_dict={}
for i in titles.index:
    itemid=titles['essay_id'][i]
    title=titles['e.c_title'][i]
    title_dict[itemid]=title

In [27]:
for deviceid,emb in user_dict.items():
    p.knn_query(emb,10)

In [141]:
n=0
print(user_data['deviceid'][n])
emb=user_data.values[n][1].split(',')
emb=[float(x) for x in emb]
for i in p.knn_query(emb,10)[0].tolist()[0]:
    itemid=effective_lst[i]
    if itemid in title_dict.keys():
        print(title_dict[itemid])

84960a1774a3460c93c5b010b4b97cef
它是高性价比摩托车,8马力110cc,满油400公里,均价仅1.2万
它是高性价比摩托车，8马力110cc，满油400公里，均价仅1.2万
车界性价之王,110cc单缸风冷,油耗仅1.6L,仅1.2万起
国人没白捧红它!一箱油开200km,125cc风冷+8马力,起步价9600
又一亲民小踏板!续航300km,单缸风冷110cc,1.2万你会买吗
实在通勤摩托车!加满油开300km,6.3kw功率110cc,起步价1.2万
摩托车界性价王！6.3kw功率110cc，1.2万你会买吗
最该火的摩托车!续航200km,搭载100cc单缸风冷,6700贵吗
这摩托车真特别!外表彰显档次,单缸风冷110cc,1.2万贵吗
又一实用踏板车!110cc风冷输出8马力,开8年难有大修,仅1.2万起


In [142]:
print(user_data['deviceid'][n])
emb=user_data.values[n][1].split(',')
emb=[float(x) for x in emb]
sim_matrix=np.dot(np.array(emb),effective_matrix.T)
sorted_list=sorted(enumerate(list(sim_matrix)), key=itemgetter(1),reverse=True)
for j,sim in sorted_list[:10]:
    simitem=effective_lst[j]
    if simitem in title_dict.keys():
        title=title_dict[simitem]
        print(title)

84960a1774a3460c93c5b010b4b97cef
俏也不争春 只把春来报 待到春花烂漫时 俺在丛中笑
它是高性价比摩托车,8马力110cc,满油400公里,均价仅1.2万
它是高性价比摩托车，8马力110cc，满油400公里，均价仅1.2万
一起来玩吧
陶冶一下情操
朴实无华   的   机车人生
车界性价之王,110cc单缸风冷,油耗仅1.6L,仅1.2万起
国人没白捧红它!一箱油开200km,125cc风冷+8马力,起步价9600
又一亲民小踏板!续航300km,单缸风冷110cc,1.2万你会买吗
实在通勤摩托车!加满油开300km,6.3kw功率110cc,起步价1.2万


In [167]:
import datetime
today=datetime.date.today() 
oneday=datetime.timedelta(days=1) 
yesterday=(today-oneday).strftime('%Y%m%d')
yesterday

'20210128'